#A Reduction Tale

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.109 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 28.266 MiB


and choose a different container for the data that we want to reduce, starting with a list:

##Regular lists

In [2]:
a = [i for i in range(1000*1000)]

In [2] used 32.516 MiB RAM in 0.168s, peaked 0.000 MiB above current, total RAM usage 60.781 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

10 loops, best of 3: 20.9 ms per loop
In [3] used 6.598 MiB RAM in 0.895s, peaked 0.000 MiB above current, total RAM usage 67.379 MiB


which, in MIPS (Mega-Instructions-Per-Second) is:

In [4]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

MIPS: 47.8
In [4] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 67.379 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

##Containers using the array module in Python

In [5]:
# Create an array of 'l'ong integers (8 bytes on 32-bit platforms)
import array
%load_ext memory_profiler

In [5] used 0.004 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 67.383 MiB


In [8]:
%memit arr = array.array('l', a)

peak memory: 80.90 MiB, increment: 0.00 MiB
In [8] used -7.848 MiB RAM in 0.541s, peaked 7.637 MiB above current, total RAM usage 73.258 MiB


7.7 ~ 15 MB vs 31 MB seems like a good deal, although sometimes Python is allocating more memory than necessary.  In fact, the array module seems to provide optimal containers from a memory consumption point of view:

In [9]:
# Size per element:
(mw.memory_delta * 2**20) / 1e6

-8.228864

In [9] used 0.000 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 73.258 MiB


But how it performs during reductions?

In [10]:
t = %timeit -o sum(arr)

10 loops, best of 3: 46.6 ms per loop
In [10] used 0.012 MiB RAM in 1.939s, peaked 0.000 MiB above current, total RAM usage 73.270 MiB


In [11]:
print("MIPS:", round(1e6 / t.best / 1e6, 1))

MIPS: 21.5
In [11] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 73.270 MiB


Well, that's a bit disappointing, as the array object performs about 2x slower than a regular array.  Not sure about the resons, but probably the array module is not getting too much attention performance-wise mainly because the NumPy existance.  Speaking of NumPy: here we go!

##NumPy

In [12]:
import numpy as np

In [12] used 10.945 MiB RAM in 0.511s, peaked 0.000 MiB above current, total RAM usage 84.215 MiB


In [13]:
na = np.array(a, dtype=np.int64)

In [13] used 7.652 MiB RAM in 0.167s, peaked 0.000 MiB above current, total RAM usage 91.867 MiB


We see that, with 8 bytes/element, NumPy is also an efficient container.

In [14]:
t = %timeit -o sum(na)

1 loops, best of 3: 201 ms per loop
In [14] used 0.023 MiB RAM in 0.859s, peaked 0.000 MiB above current, total RAM usage 91.891 MiB


In [15]:
print("MIPS:", round(1e6 / t.best / 1e6, 3))

MIPS: 4.979
In [15] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 91.891 MiB


Oops, this is more than several times slower than the `array` module.  Why so?

**Answer:** NumPy has a lot of overhead in producing a Python integer for every element in the array.

*Hint:* Use internal methods (ufuncs) when possible.

In [16]:
t = %timeit -o na.sum()

The slowest run took 4.89 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 1.73 ms per loop
In [16] used 0.070 MiB RAM in 7.392s, peaked 0.000 MiB above current, total RAM usage 91.961 MiB


In [17]:
print("MIPS:", round(1e6 / t.best / 1e6, 3))

MIPS: 577.688
In [17] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 91.961 MiB


This is about 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

##Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum memory speed that supports your laptop?

##Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [18]:
import bcolz

In [18] used 22.391 MiB RAM in 1.545s, peaked 0.000 MiB above current, total RAM usage 114.352 MiB


In [19]:
ca = bcolz.carray(na)

In [19] used 0.348 MiB RAM in 0.019s, peaked 0.000 MiB above current, total RAM usage 114.699 MiB


In [20]:
print("mem_used:", mw.measurements.memory_delta)

mem_used: 0.34765625
In [20] used 0.027 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 114.727 MiB


Ok, this time the amount of memory used seems much lower.  Also, bcolz containers can provide an estimation on how much memory they are taking; let's have a look:

In [22]:
ca

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 394.28 KB; ratio: 19.81
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [22] used 0.004 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 114.906 MiB


In this case we see that bcolz estimation is very close to `ipython_memwatcher` measurements.  Let's have a look at the speed of the reduction:

In [23]:
t = %timeit -o ca.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

100 loops, best of 3: 4.7 ms per loop
MIPS: 212.891
In [23] used -89.328 MiB RAM in 2.023s, peaked 0.000 MiB above current, total RAM usage 25.578 MiB


This is around 2~4x slower (depending on the machine) than a regular NumPy array, but the size of the array is an impressive 20x smaller.  But is compression the only responsible of the overhead?  Let's investigate a bit further.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [24]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [24] used 54.441 MiB RAM in 0.357s, peaked 0.000 MiB above current, total RAM usage 80.020 MiB


In [25]:
cau

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 7.75 MB; ratio: 0.98
  cparams := cparams(clevel=0, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [25] used 0.668 MiB RAM in 0.005s, peaked 0.000 MiB above current, total RAM usage 80.688 MiB


In [26]:
t = %timeit -o cau.sum()
print("MIPS:", round(1e6 / t.best / 1e6, 3))

100 loops, best of 3: 4.1 ms per loop
MIPS: 244.027
In [26] used 0.262 MiB RAM in 1.746s, peaked 0.000 MiB above current, total RAM usage 80.949 MiB


As we can see, the times with an uncompressed `carray` are very close to a compressed one, so compressing is not the only source of the overhead (and it is very minor in fact).

So, bcolz allows to use compressed in-memory data containers at the cost of some performance (compared with NumPy).  But the performance overhead is rather small, and sometimes you prefer to keep more data in-memory.

On another hand, in the next notebooks we are going to see that bcolz can be competitive with NumPy, performance wise, in some scenarios.

### Exercise: Using bcolz in real scenarios

bcolz does not get good compression ratios only with synthetic data, but with real data too.  Be sure to check out this URL:

http://nbviewer.ipython.org/gist/alimanfoo/e93a532eb6bde311ea39/genotype_bitshuffle.ipynb

and let's discuss this specific case of bcolz usage in genomics:

* Which are the typical compression ratios for this case?

* Is there a difference in speed accessing data in compressed and non-compressed state (clevel=0)

* Which are the compressors achieving the best compression/speed ratio?